**PySDS Week 2 Lecture 4. V.1 **
Last author: B. Hogan

Week 2 Day 4. : Merging and grouping data 
=================

This week we are going to focus primarily on exercises where you must integrate different data sources together in a single table for analysis. 

Learning goals: 
- Understand merging / sorting
- Be able to read and write a table from iPython
- Understand one-to-many and many-to-many relationships. 
- Understanding grouping relationships
- Use a simple pivot table

# Section 1. A review of adding data to a DataFrame

First, let's revisit the merging of data through append and concatenate and then move on to key-based merging. 

First we will create two dataframes based on dictionaries, then we will combine them. We will do this in two ways: 
1. The same columns (adding rows) 
2. The same rows (adding columns) 

## Adding rows 
When adding data where we have the same columns, it is typically because we have new rows. This happens when we are processing data and want to add rows one at a time as the data comes in. You have seen this already.

*Things to remember:* 
- DataFrames have rows, and each row has an index. 
- The index can have a user-defined value, but it is assigned in numerical sequence by default. 

In [41]:
from pandas import Series, DataFrame
import pandas as pd 
import numpy as np
from IPython.display import display
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [43]:

testData1 = [["a","b","c","d"],["g","h","j","k"]]
testFrame1 = pd.DataFrame(testData1)
print(testFrame1)

print()

testData2 = [["m","n","o","p"],["s","t","u","v"]]
testFrame2 = pd.DataFrame(testData2)
print(testFrame2)

print()

testData3 = [["x","y","z","aa","bb","cc"],["e","f","q","w","ww","www"]]
testFrame3 = pd.DataFrame(testData3)
print(testFrame3)




   0  1  2  3
0  a  b  c  d
1  g  h  j  k

   0  1  2  3
0  m  n  o  p
1  s  t  u  v

   0  1  2   3   4    5
0  x  y  z  aa  bb   cc
1  e  f  q   w  ww  www


### Attempt 1: Adding the frames together ###

In the first case, see what happens when we add the frames together. Because they are the same dimension, it literally concatenates within cell. If the cells are not the same size, they will repeat. See the two results below. 

In [44]:
exData01 = testData1 + testData2
display(exData01)

print()

# Notice the difference between adding the lists and adding the frames. 
exFrame01 = testFrame1 + testFrame2
display(exFrame01)

print()

# Notice now the software does not know what to do adding misshapen frames. 
exFrame02 = testFrame1 + testFrame3
display(exFrame02)

[['a', 'b', 'c', 'd'],
 ['g', 'h', 'j', 'k'],
 ['m', 'n', 'o', 'p'],
 ['s', 't', 'u', 'v']]

,0,1,2,3
0,am,bn,co,dp
1,gs,ht,ju,kv


,0,1,2,3,4,5
0,ax,by,cz,daa,NaN,NaN
1,ge,hf,jq,kw,NaN,NaN


### Attempt 2: Concatenating frames ###
In the second case, we are going to concatenate the data. The first way we will be doing this is by row. Recall what happens to the indices by default.

In [45]:
testFrame4 = pd.concat(    [testFrame1, testFrame2])
testFrame4

,0,1,2,3
0,a,b,c,d
1,g,h,j,k
0,m,n,o,p
1,s,t,u,v


In [7]:
# To really understand the method, it's useful to read the help file. 
help(pd.concat)

Help on function concat in module pandas.core.reshape.concat:

concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, sort=None, copy=True)
    Concatenate pandas objects along a particular axis with optional set logic
    along the other axes.
    
    Can also add a layer of hierarchical indexing on the concatenation axis,
    which may be useful if the labels are the same (or overlapping) on
    the passed axis number.
    
    Parameters
    ----------
    objs : a sequence or mapping of Series, DataFrame, or Panel objects
        If a dict is passed, the sorted keys will be used as the `keys`
        argument, unless it is passed, in which case the values will be
        selected (see below). Any None objects will be dropped silently unless
        they are all None in which case a ValueError will be raised
    axis : {0/'index', 1/'columns'}, default 0
        The axis to concatenate along
    join : {'in

Below we fix this using the *ignore_index = True* argument.  

Notice also that pd.concat and DataFrame.append accomplish the same thing but are not implemented the same way. Generally concat is faster.

In [46]:
testFrame4 = pd.concat([testFrame1, testFrame2],ignore_index=True)
print(testFrame4)

print()

testFrame4 = testFrame1.append(testFrame2,ignore_index=True)
print(testFrame4)

   0  1  2  3
0  a  b  c  d
1  g  h  j  k
2  m  n  o  p
3  s  t  u  v

   0  1  2  3
0  a  b  c  d
1  g  h  j  k
2  m  n  o  p
3  s  t  u  v


Now if we want to add these as **columns rather than rows**, we can use the *axis=1* (as opposed to the default axis=0 argument)

Also notice that this is not available as appending.

In [47]:
testFrame4 = pd.concat([testFrame1, testFrame2],axis=1)
testFrame4.index = ['top', 'bottom']
display(testFrame4)

print()

,0,1,2,3,0,1,2,3
top,a,b,c,d,m,n,o,p
bottom,g,h,j,k,s,t,u,v


In [48]:
testFrame4.loc["top",0] = "test"
display(testFrame4)

,0,1,2,3,0,1,2,3
top,test,b,c,d,test,n,o,p
bottom,g,h,j,k,s,t,u,v


If we want to preservethat index for some reason, we can actually use a multi-index. This is where there are subindices for the dataframe. This is also relevant when you are grouping data, as the grouped data can have a multi-index. 

In [50]:
testFrame4 = pd.concat([testFrame1, testFrame2],axis=1,keys=["left","right"])
testFrame4.index = ['top', 'bottom']
display(testFrame4)

print()

left          right         
          0  1  2  3     0  1  2  3
top       a  b  c  d     m  n  o  p
bottom    g  h  j  k     s  t  u  v

In [58]:
print(testFrame4["left",0])
print(testFrame4["left",0]['top'])
try: 
    print(testFrame4["left",0,'top'])
except KeyError:
    print("The first bracket is for the index only.")

top       a
bottom    g
Name: (left, 0), dtype: object
a
The first bracket is for the index only.


If we want to **add a single series**, then we have to be careful about 
how it is structured. Noticed in the following. We can see this being done right and wrong. 

In [59]:
testSeries1 = pd.Series(["alpha","bravo","charlie","delta"],name="example")

testFrame5 = testFrame1.append(testSeries1)#,ignore_index=True)
testFrame5

,0,1,2,3
0,a,b,c,d
1,g,h,j,k
example,alpha,bravo,charlie,delta


In [60]:
testSeries1 = pd.Series({2:"bravo",3:"charlie",4:"delta",1:"alpha"},name="example")

testFrame5 = testFrame1.append(testSeries1)
testFrame5

,0,1,2,3,4
0,a,b,c,d,NaN
1,g,h,j,k,NaN
example,NaN,alpha,bravo,charlie,delta


In [61]:
# Ooops! It's "Zero" indexed

testSeries1 = pd.Series({0:"alpha",1:"bravo",2:"charlie",3:"delta"},name="example")

testFrame5 = testFrame1.append(testSeries1)
testFrame5

,0,1,2,3
0,a,b,c,d
1,g,h,j,k
example,alpha,bravo,charlie,delta


## Addding Columns 

Each DataFrame has an index and a series of columns. To add names to the index, you can assign a variable to DataFrame.index. To assign names to the columns, you can use DataFrame.columns. These are lists. They cannot be shorter or longer than the actual data frame, otherwise you will receive a ValueError. 

In [62]:
testFrame5.columns = ["first","second","third","fourth"]
display(testFrame5)

print(len(testFrame5.columns))

try:
    testFrame5.columns = ["1first","2second","3third"]
    display(testFrame5)
except ValueError:
    print("ValueError: Length mismatch")
    

testFrame5.index = ["first_row","second_row","third_row"]
display(testFrame5)

try:
    testFrame5.index = ["first_row","second_row","third_row","fourth_row"]
    display(testFrame5)
except ValueError:
    print("ValueError: Length mismatch")
    

    

,first,second,third,fourth
0,a,b,c,d
1,g,h,j,k
example,alpha,bravo,charlie,delta


4
ValueError: Length mismatch


,first,second,third,fourth
first_row,a,b,c,d
second_row,g,h,j,k
third_row,alpha,bravo,charlie,delta


ValueError: Length mismatch


# One-to-many relationships

One to many relationships are really common in data wrangling. For example, you have people who are in states, and you have state level data on unemployment. How do you create a new table that includes these state-level indicators? This might be useful for a regression (particularly a popular class of regression models called 'hierarchical linear models'). 

In [65]:
d = {"Wales":3,"England":53,"Scotland":5,"Northern Ireland":2,"Jersey":.1}
l = list(zip(d.keys(),d.values()))
print(l)

countryFrame = pd.DataFrame(l,columns=["Country","Population"])
display(countryFrame)

countryFrame = pd.DataFrame(pd.Series(d),columns=["Population"])
countryFrame

[('Wales', 3), ('England', 53), ('Scotland', 5), ('Northern Ireland', 2), ('Jersey', 0.1)]


,Country,Population
0,Wales,3.0
1,England,53.0
2,Scotland,5.0
3,Northern Ireland,2.0
4,Jersey,0.1


,Population
Wales,3.0
England,53.0
Scotland,5.0
Northern Ireland,2.0
Jersey,0.1


In [22]:
people = [["Alice",32,"Wales"],
          ["Bob",35,"Northern Ireland"],
          ["Charlie",21,"England"],
          ["Diane",45,"Northern Ireland"],
          ["Ellen",21,"Scotland"],
          ["Fong",50,"England"],
          ["Grant",28,"Scotland"],
          ["Harry",36,"England"],
          ["Idris",40,"Isle of Wight"]]

peopleFrame = pd.DataFrame(people,columns=["Name","Age","Country"])
# help(peopleFrame.merge)
display(peopleFrame)

,Name,Age,Country
0,Alice,32,Wales
1,Bob,35,Northern Ireland
2,Charlie,21,England
3,Diane,45,Northern Ireland
4,Ellen,21,Scotland
5,Fong,50,England
6,Grant,28,Scotland
7,Harry,36,England
8,Idris,40,Isle of Wight


In [23]:
mergeFrame = peopleFrame.merge(countryFrame,left_on="Country",right_index=True)
display(mergeFrame)

mergeFrame = countryFrame.merge(peopleFrame,left_index=True,right_on="Country")
display(mergeFrame)

,Name,Age,Country,Population
0,Alice,32,Wales,3.0
1,Bob,35,Northern Ireland,2.0
3,Diane,45,Northern Ireland,2.0
2,Charlie,21,England,53.0
5,Fong,50,England,53.0
7,Harry,36,England,53.0
4,Ellen,21,Scotland,5.0
6,Grant,28,Scotland,5.0


,Population,Name,Age,Country
0,3.0,Alice,32,Wales
2,53.0,Charlie,21,England
5,53.0,Fong,50,England
7,53.0,Harry,36,England
4,5.0,Ellen,21,Scotland
6,5.0,Grant,28,Scotland
1,2.0,Bob,35,Northern Ireland
3,2.0,Diane,45,Northern Ireland


So. Merging / Joining is REALLY hard to get your head around. And it won't necessarily work the first couple times (I went through several iterations in getting the examples to work). But let's give a little overview of ways to join:

- Left: Unique rows on the left, mutliple on the right. 
- Right: Unique rows on the right, multiple on the left.
- Inner: The intersection of both frames.
- Outer: The union of both frames. 

Below is a very small crash course in "Union" and "Intersection". 

In [24]:
setleft = set([1,3,5,7,9])
setright = set([1,2,3,4,5])
print("Union: PRINT ALL THE THINGS!")
print(setleft.union(setright))

print("\nIntersection: Here's what we have in common")
print(setleft.intersection(setright))

Union: PRINT ALL THE THINGS!
{1, 2, 3, 4, 5, 7, 9}

Intersection: Here's what we have in common
{1, 3, 5}


Now let's return to the data we analyzed above and explore what happens when we join in different ways. 

### Outer Join

Notice above we include both Jersey and Isle of Wight, and then get some missing data. Country_y (which we really ought to rename) is missing for Isle of Wight and the individuals are missing for Jersey. It's the union of the keys. 

In [25]:
mergeFrame = peopleFrame.merge(countryFrame,left_on="Country",right_index=True, how='outer')
mergeFrame.reset_index(drop=True)

,Name,Age,Country,Population
0,Alice,32.0,Wales,3.0
1,Bob,35.0,Northern Ireland,2.0
2,Diane,45.0,Northern Ireland,2.0
3,Charlie,21.0,England,53.0
4,Fong,50.0,England,53.0
5,Harry,36.0,England,53.0
6,Ellen,21.0,Scotland,5.0
7,Grant,28.0,Scotland,5.0
8,Idris,40.0,Isle of Wight,NaN
9,NaN,NaN,Jersey,0.1


### Inner Join

Notice in this case, just like with an intersection, we get rid of the keys where there is no match in the other table. So, goodbye Isle of Wight and goodbye Jersey! 

In [26]:
mergeFrame = peopleFrame.merge(countryFrame,left_on="Country",right_index=True, how='inner')
display(mergeFrame)

,Name,Age,Country,Population
0,Alice,32,Wales,3.0
1,Bob,35,Northern Ireland,2.0
3,Diane,45,Northern Ireland,2.0
2,Charlie,21,England,53.0
5,Fong,50,England,53.0
7,Harry,36,England,53.0
4,Ellen,21,Scotland,5.0
6,Grant,28,Scotland,5.0


### Left Join

Notice above that we joined on left (which is "peopleFrame"). We could have also done the following, which would have been roughly equivalent (see for yourself!)

    pd.merge(peopleFrame,countryFrame,left_on="Country",right_index=True, how='left')

    peopleFrame.join(countryFrame,on="Country",how="inner",rsuffix="_x")
    
I say roughly equivalent, because the join command is actually a little more tidy than merge. Notice that the inner join doesn't have country_x and country_y, but merges those. In the end, no way is particularly "correct".

In [27]:
mergeFrame = peopleFrame.merge(countryFrame,left_on="Country",right_index=True, how='left')
mergeFrame

,Name,Age,Country,Population
0,Alice,32,Wales,3.0
1,Bob,35,Northern Ireland,2.0
2,Charlie,21,England,53.0
3,Diane,45,Northern Ireland,2.0
4,Ellen,21,Scotland,5.0
5,Fong,50,England,53.0
6,Grant,28,Scotland,5.0
7,Harry,36,England,53.0
8,Idris,40,Isle of Wight,NaN


In [28]:
countryFrame.columns = ["Country"]
display(countryFrame)

,Country
Wales,3.0
England,53.0
Scotland,5.0
Northern Ireland,2.0
Jersey,0.1


In [29]:
# countryFrame.columns = ["Country"]
peopleFrame.join(countryFrame,on="Country",how="inner",rsuffix="_rightTable")

,Name,Age,Country,Country_rightTable
0,Alice,32,Wales,3.0
1,Bob,35,Northern Ireland,2.0
3,Diane,45,Northern Ireland,2.0
2,Charlie,21,England,53.0
5,Fong,50,England,53.0
7,Harry,36,England,53.0
4,Ellen,21,Scotland,5.0
6,Grant,28,Scotland,5.0


### Right Join

Pretty much the same as the left join, except it is merging on the right instead of the left. 

In [30]:
mergeFrame = peopleFrame.merge(countryFrame,left_on="Country",right_index=True, how='right')
mergeFrame

,Country,Name,Age,Country_x,Country_y
0,Wales,Alice,32.0,Wales,3.0
1,Northern Ireland,Bob,35.0,Northern Ireland,2.0
3,Northern Ireland,Diane,45.0,Northern Ireland,2.0
2,England,Charlie,21.0,England,53.0
5,England,Fong,50.0,England,53.0
7,England,Harry,36.0,England,53.0
4,Scotland,Ellen,21.0,Scotland,5.0
6,Scotland,Grant,28.0,Scotland,5.0
8,Jersey,NaN,NaN,NaN,0.1


** Multiple columns ** 

You'll notice that in the above example, countryFrame is really just one column of data. Let's add another column to it, and see what happens to our merging.

In [31]:
countryFrame = pd.DataFrame([["Wales",3,28],
                            ["England",53,51],
                            ["Scotland",5,46],
                            ["Northern Ireland",2,27],
                            ["Jersey",.1,58]],
                            columns=["Country","Population","Income"])
display(countryFrame)

,Country,Population,Income
0,Wales,3.0,28
1,England,53.0,51
2,Scotland,5.0,46
3,Northern Ireland,2.0,27
4,Jersey,0.1,58


Now in this case, you'll notice that we no longer have the names of the countries as indices, so we will have to change the way we merge ever so slightly. 

In [32]:
mergeFrame = peopleFrame.merge(countryFrame, on="Country")
mergeFrame

,Name,Age,Country,Population,Income
0,Alice,32,Wales,3.0,28
1,Bob,35,Northern Ireland,2.0,27
2,Diane,45,Northern Ireland,2.0,27
3,Charlie,21,England,53.0,51
4,Fong,50,England,53.0,51
5,Harry,36,England,53.0,51
6,Ellen,21,Scotland,5.0,46
7,Grant,28,Scotland,5.0,46


Grouping Data 
===================

Broadcasting Aggregations back to the original data
---------------------------

Now with these columns imagine that we want to create some variable that is a group-level aggregation of individual level variables. Population and Income are already group-level variables . 

We can group data together using the 

    groupby(KEY) 
    
command. First we will group all the data, and then just a subset of it. 

In [33]:
groupFrame = mergeFrame.groupby('Country').sum()
display(groupFrame)

,Age,Population,Income
Country,,,
England,107,159.0,153
Northern Ireland,80,4.0,54
Scotland,49,10.0,92
Wales,32,3.0,28


In [34]:
help(mergeFrame.groupby)

Help on method groupby in module pandas.core.generic:

groupby(by=None, axis=0, level=None, as_index=True, sort=True, group_keys=True, squeeze=False, observed=False, **kwargs) method of pandas.core.frame.DataFrame instance
    Group series using mapper (dict or key function, apply given function
    to group, return result as series) or by a series of columns.
    
    Parameters
    ----------
    by : mapping, function, label, or list of labels
        Used to determine the groups for the groupby.
        If ``by`` is a function, it's called on each value of the object's
        index. If a dict or Series is passed, the Series or dict VALUES
        will be used to determine the groups (the Series' values are first
        aligned; see ``.align()`` method). If an ndarray is passed, the
        values are used as-is determine the groups. A label or list of
        labels may be passed to group by the columns in ``self``. Notice
        that a tuple is interpreted a (single) key.
    a

You'll notice that it adds the prefix "m\_" to all the scalar values and uses all of them. This is alright, but if we want to merge these values back into the original data set, this will be a nuisance since m_Population is the same as Population since both came from the Country table to begin with. So, we can group on a slice of the dataframe. To slice the dataframe we have to query it in the following way. 

    DATAFRAME[ ['VAR1','VAR2'] ]
    
Yes, that's a list within a list. See below:    

In [35]:
mergeFrame[["Country","Age","Income"]]

,Country,Age,Income
0,Wales,32,28
1,Northern Ireland,35,27
2,Northern Ireland,45,27
3,England,21,51
4,England,50,51
5,England,36,51
6,Scotland,21,46
7,Scotland,28,46


In [36]:
groupFrame = mergeFrame[["Country","Age"]].groupby('Country').mean()

groupFrame

,Age
Country,
England,35.666667
Northern Ireland,40.000000
Scotland,24.500000
Wales,32.000000


We have just one issue now - if we merge the average age back in, there is already a variable is already called Age. We can rename it before we merge it back in, but it is easier to add a prefix when we do the original grouping:

    DATAFRAME.groupby(KEY).add_prefix("mean_")
    

In [37]:
groupFrame = mergeFrame[["Country","Age"]].groupby('Country').mean().add_prefix("mean_")
groupFrame

,mean_Age
Country,
England,35.666667
Northern Ireland,40.000000
Scotland,24.500000
Wales,32.000000


In [38]:
newFrame = pd.merge(mergeFrame, groupFrame,left_on="Country",right_index=True)
display(newFrame)

,Name,Age,Country,Population,Income,mean_Age
0,Alice,32,Wales,3.0,28,32.000000
1,Bob,35,Northern Ireland,2.0,27,40.000000
2,Diane,45,Northern Ireland,2.0,27,40.000000
3,Charlie,21,England,53.0,51,35.666667
4,Fong,50,England,53.0,51,35.666667
5,Harry,36,England,53.0,51,35.666667
6,Ellen,21,Scotland,5.0,46,24.500000
7,Grant,28,Scotland,5.0,46,24.500000


Copying versus addressing
-------------------------

So, is m_Age part of the mergeFrame table now? No! Part of the way that Pandas works is to only put things in memory unless otherwise stated. See below:


In [39]:
newFrame["age_meancentered"] = newFrame["mean_Age"] - newFrame["Age"]
newFrame

,Name,Age,Country,Population,Income,mean_Age,age_meancentered
0,Alice,32,Wales,3.0,28,32.000000,0.000000
1,Bob,35,Northern Ireland,2.0,27,40.000000,5.000000
2,Diane,45,Northern Ireland,2.0,27,40.000000,-5.000000
3,Charlie,21,England,53.0,51,35.666667,14.666667
4,Fong,50,England,53.0,51,35.666667,-14.333333
5,Harry,36,England,53.0,51,35.666667,-0.333333
6,Ellen,21,Scotland,5.0,46,24.500000,3.500000
7,Grant,28,Scotland,5.0,46,24.500000,-3.500000


In [40]:
mergeFrame = pd.merge(mergeFrame, groupFrame,left_on="Country",right_index=True)
mergeFrame

# Ta-dah! 

,Name,Age,Country,Population,Income,mean_Age
0,Alice,32,Wales,3.0,28,32.000000
1,Bob,35,Northern Ireland,2.0,27,40.000000
2,Diane,45,Northern Ireland,2.0,27,40.000000
3,Charlie,21,England,53.0,51,35.666667
4,Fong,50,England,53.0,51,35.666667
5,Harry,36,England,53.0,51,35.666667
6,Ellen,21,Scotland,5.0,46,24.500000
7,Grant,28,Scotland,5.0,46,24.500000


In [ ]:
# Split-apply-combine 

